In [1]:
from __future__ import print_function, division
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import data as dt
import model_4 as md
import copy
from pytorch_ssim import ssim
from torch.utils.data import DataLoader
# e/d + i + up

In [2]:
# print gpu
torch.cuda.set_device(6)
currentDevice = torch.cuda.current_device()
print("Current GPU: " + str(currentDevice))
print(str(torch.cuda.device_count()))
print(str(torch.cuda.get_device_capability(currentDevice)))
print(torch.__version__)

Current GPU: 6
8
(6, 1)
1.0.0


In [3]:
USE_GPU = 1
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda:6')
else:
    device = "cpu"
print(device)

cuda:6


In [ ]:
# generate csv file, run only for the first time
# dt.generate_csv()

In [4]:
csvFilePath = dt.get_csv_path()
transformed_dataset = dt.HE_SHG_Dataset(csv_file=csvFilePath,
                                               transform=dt.Compose([                                              
                                               dt.Rescale(96),                                     
                                               dt.Normalize(),
                                               dt.ToTensor()
                                           ]))
# TODO: change the normalization parameters

In [5]:
# batchsize 32->16
dataloader = DataLoader(transformed_dataset, batch_size=32,
                        shuffle=True, num_workers=0)

In [ ]:
# TODO: insert back mean and variance to plot the image appropriately
dt.show_patch(dataloader) 

In [6]:
print('===> Building model')
model = md.Net().to(device)
criterionMSE = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)


===> Building model


In [7]:
def train(epoch):
    epoch_loss = 0
    for iteration, batch in enumerate(dataloader):
        input, target = batch['input'].to(device), batch['output'].to(device)

        optimizer.zero_grad()
        output = model(input)
        targetf = target.float()
        targetf = targetf[:, None]
        
        lossMSE = criterionMSE(output, targetf)      
        lossSSIM = -(ssim(output, targetf)-1)
        
        p = 0.75
        loss = p*lossMSE + (1-p)*lossSSIM
        combineLoss = p*lossMSE.item() + (1-p)*lossSSIM.item()
        
        epoch_loss = epoch_loss + combineLoss
        loss.backward()
        optimizer.step()
    
        if iteration%50 == 0:
            print("lossMSE: " + str(lossMSE.item()) +
                  " " + "lossSSIM: " + str(lossSSIM.item()))
            print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(dataloader), loss.item()))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(dataloader)))


In [ ]:
# def test():
#     avg_psnr = 0
#     with torch.no_grad():
#         for batch in testing_data_loader:
#             input, target = batch[0].to(device), batch[1].to(device)

#             prediction = model(input)
#             mse = criterion(prediction, target)
#             psnr = 10 * log10(1 / mse.item())
#             avg_psnr += psnr
#     print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))


In [ ]:
# def checkpoint(epoch):
#     model_out_path = "model_epoch_{}.pth".format(epoch)
#     torch.save(model, model_out_path)
#     print("Checkpoint saved to {}".format(model_out_path))

In [8]:
for epoch in range(1, 10 + 1):
    train(epoch)
#     test()
#     checkpoint(epoch)
#     wanna loss < 0.06

lossMSE: 0.257510781288147 lossSSIM: 0.974213182926178
===> Epoch[1](0/25343): Loss: 0.4367
lossMSE: 0.22467820346355438 lossSSIM: 0.9554298520088196
===> Epoch[1](50/25343): Loss: 0.4074
lossMSE: 0.22435900568962097 lossSSIM: 0.9483346343040466
===> Epoch[1](100/25343): Loss: 0.4054
lossMSE: 0.22294312715530396 lossSSIM: 0.9519307017326355
===> Epoch[1](150/25343): Loss: 0.4052
lossMSE: 0.21854645013809204 lossSSIM: 0.9476061463356018
===> Epoch[1](200/25343): Loss: 0.4008
lossMSE: 0.22295920550823212 lossSSIM: 0.9513169527053833
===> Epoch[1](250/25343): Loss: 0.4050
lossMSE: 0.2296634465456009 lossSSIM: 0.9584789276123047
===> Epoch[1](300/25343): Loss: 0.4119
lossMSE: 0.22208182513713837 lossSSIM: 0.9483994245529175
===> Epoch[1](350/25343): Loss: 0.4037
lossMSE: 0.2211720198392868 lossSSIM: 0.9518765807151794
===> Epoch[1](400/25343): Loss: 0.4038
lossMSE: 0.21857109665870667 lossSSIM: 0.954988420009613
===> Epoch[1](450/25343): Loss: 0.4027
lossMSE: 0.21024982631206512 lossSSIM: 

lossMSE: 0.14557772874832153 lossSSIM: 0.9334552884101868
===> Epoch[1](4250/25343): Loss: 0.3425
lossMSE: 0.15064267814159393 lossSSIM: 0.9507074952125549
===> Epoch[1](4300/25343): Loss: 0.3507
lossMSE: 0.14308620989322662 lossSSIM: 0.9336806535720825
===> Epoch[1](4350/25343): Loss: 0.3407
lossMSE: 0.13381056487560272 lossSSIM: 0.912666380405426
===> Epoch[1](4400/25343): Loss: 0.3285
lossMSE: 0.14503179490566254 lossSSIM: 0.9423224329948425
===> Epoch[1](4450/25343): Loss: 0.3444
lossMSE: 0.14548340439796448 lossSSIM: 0.9421865344047546
===> Epoch[1](4500/25343): Loss: 0.3447
lossMSE: 0.14226330816745758 lossSSIM: 0.939926266670227
===> Epoch[1](4550/25343): Loss: 0.3417
lossMSE: 0.14255136251449585 lossSSIM: 0.9382586479187012
===> Epoch[1](4600/25343): Loss: 0.3415
lossMSE: 0.13315711915493011 lossSSIM: 0.9231044054031372
===> Epoch[1](4650/25343): Loss: 0.3306
lossMSE: 0.13391193747520447 lossSSIM: 0.9288336038589478
===> Epoch[1](4700/25343): Loss: 0.3326
lossMSE: 0.13770638406

lossMSE: 0.08990904688835144 lossSSIM: 0.9275224208831787
===> Epoch[1](8450/25343): Loss: 0.2993
lossMSE: 0.08226514607667923 lossSSIM: 0.9023692607879639
===> Epoch[1](8500/25343): Loss: 0.2873
lossMSE: 0.08820901811122894 lossSSIM: 0.9228771924972534
===> Epoch[1](8550/25343): Loss: 0.2969
lossMSE: 0.08647548407316208 lossSSIM: 0.9174479246139526
===> Epoch[1](8600/25343): Loss: 0.2942
lossMSE: 0.08490481972694397 lossSSIM: 0.9184607863426208
===> Epoch[1](8650/25343): Loss: 0.2933
lossMSE: 0.08445879817008972 lossSSIM: 0.9234194755554199
===> Epoch[1](8700/25343): Loss: 0.2942
lossMSE: 0.08598333597183228 lossSSIM: 0.9257214069366455
===> Epoch[1](8750/25343): Loss: 0.2959
lossMSE: 0.08235544711351395 lossSSIM: 0.914383053779602
===> Epoch[1](8800/25343): Loss: 0.2904
lossMSE: 0.08026110380887985 lossSSIM: 0.9066331386566162
===> Epoch[1](8850/25343): Loss: 0.2869
lossMSE: 0.08387848734855652 lossSSIM: 0.9207484126091003
===> Epoch[1](8900/25343): Loss: 0.2931
lossMSE: 0.0795437991

lossMSE: 0.05037983879446983 lossSSIM: 0.8984870910644531
===> Epoch[1](12650/25343): Loss: 0.2624
lossMSE: 0.05056647211313248 lossSSIM: 0.9046555757522583
===> Epoch[1](12700/25343): Loss: 0.2641
lossMSE: 0.04708849638700485 lossSSIM: 0.8812409043312073
===> Epoch[1](12750/25343): Loss: 0.2556
lossMSE: 0.049934107810258865 lossSSIM: 0.9005227088928223
===> Epoch[1](12800/25343): Loss: 0.2626
lossMSE: 0.04900876432657242 lossSSIM: 0.9041290879249573
===> Epoch[1](12850/25343): Loss: 0.2628
lossMSE: 0.048598580062389374 lossSSIM: 0.8938947319984436
===> Epoch[1](12900/25343): Loss: 0.2599
lossMSE: 0.04780013859272003 lossSSIM: 0.8875126838684082
===> Epoch[1](12950/25343): Loss: 0.2577
lossMSE: 0.048368439078330994 lossSSIM: 0.8936895728111267
===> Epoch[1](13000/25343): Loss: 0.2597
lossMSE: 0.04742121696472168 lossSSIM: 0.8929160833358765
===> Epoch[1](13050/25343): Loss: 0.2588
lossMSE: 0.0480317622423172 lossSSIM: 0.905325174331665
===> Epoch[1](13100/25343): Loss: 0.2624
lossMSE: 

lossMSE: 0.028687499463558197 lossSSIM: 0.865081787109375
===> Epoch[1](16800/25343): Loss: 0.2378
lossMSE: 0.0296261515468359 lossSSIM: 0.8701324462890625
===> Epoch[1](16850/25343): Loss: 0.2398
lossMSE: 0.02840770035982132 lossSSIM: 0.8712279796600342
===> Epoch[1](16900/25343): Loss: 0.2391
lossMSE: 0.029607508331537247 lossSSIM: 0.8797971606254578
===> Epoch[1](16950/25343): Loss: 0.2422
lossMSE: 0.029842667281627655 lossSSIM: 0.8748744130134583
===> Epoch[1](17000/25343): Loss: 0.2411
lossMSE: 0.028645038604736328 lossSSIM: 0.8597215414047241
===> Epoch[1](17050/25343): Loss: 0.2364
lossMSE: 0.02703441120684147 lossSSIM: 0.8440109491348267
===> Epoch[1](17100/25343): Loss: 0.2313
lossMSE: 0.02609025500714779 lossSSIM: 0.8429989814758301
===> Epoch[1](17150/25343): Loss: 0.2303
lossMSE: 0.028664473444223404 lossSSIM: 0.8610929250717163
===> Epoch[1](17200/25343): Loss: 0.2368
lossMSE: 0.02873387560248375 lossSSIM: 0.8694311380386353
===> Epoch[1](17250/25343): Loss: 0.2389
lossMSE

...

lossMSE: 0.014013250358402729 lossSSIM: 0.5451797246932983
===> Epoch[2](3900/25343): Loss: 0.1468
lossMSE: 0.01380410697311163 lossSSIM: 0.5698050260543823
===> Epoch[2](3950/25343): Loss: 0.1528
lossMSE: 0.012950518168509007 lossSSIM: 0.5540190935134888
===> Epoch[2](4000/25343): Loss: 0.1482
lossMSE: 0.014465740881860256 lossSSIM: 0.5646224021911621
===> Epoch[2](4050/25343): Loss: 0.1520
lossMSE: 0.013747867196798325 lossSSIM: 0.5363041758537292
===> Epoch[2](4100/25343): Loss: 0.1444
lossMSE: 0.014837907627224922 lossSSIM: 0.5630749464035034
===> Epoch[2](4150/25343): Loss: 0.1519
lossMSE: 0.013588718138635159 lossSSIM: 0.5253045558929443
===> Epoch[2](4200/25343): Loss: 0.1415
lossMSE: 0.0164911150932312 lossSSIM: 0.5420103073120117
===> Epoch[2](4250/25343): Loss: 0.1479
lossMSE: 0.0172710083425045 lossSSIM: 0.5374320149421692
===> Epoch[2](4300/25343): Loss: 0.1473
lossMSE: 0.014524970203638077 lossSSIM: 0.5795555114746094
===> Epoch[2](4350/25343): Loss: 0.1558
lossMSE: 0.0122

lossMSE: 0.012282650917768478 lossSSIM: 0.438448965549469
===> Epoch[2](8050/25343): Loss: 0.1188
lossMSE: 0.012787897139787674 lossSSIM: 0.4765903949737549
===> Epoch[2](8100/25343): Loss: 0.1287
lossMSE: 0.01686498522758484 lossSSIM: 0.4750516414642334
===> Epoch[2](8150/25343): Loss: 0.1314
lossMSE: 0.01249187346547842 lossSSIM: 0.4603830575942993
===> Epoch[2](8200/25343): Loss: 0.1245
lossMSE: 0.011730530299246311 lossSSIM: 0.4482613801956177
===> Epoch[2](8250/25343): Loss: 0.1209
lossMSE: 0.017757587134838104 lossSSIM: 0.45350807905197144
===> Epoch[2](8300/25343): Loss: 0.1267
lossMSE: 0.011804274283349514 lossSSIM: 0.4698442816734314
===> Epoch[2](8350/25343): Loss: 0.1263
lossMSE: 0.014129082672297955 lossSSIM: 0.4537167549133301
===> Epoch[2](8400/25343): Loss: 0.1240
lossMSE: 0.012684978544712067 lossSSIM: 0.4720578193664551
===> Epoch[2](8450/25343): Loss: 0.1275
lossMSE: 0.01887454278767109 lossSSIM: 0.44573187828063965
===> Epoch[2](8500/25343): Loss: 0.1256
lossMSE: 0.0

lossMSE: 0.006697370670735836 lossSSIM: 0.3612215518951416
===> Epoch[2](12200/25343): Loss: 0.0953
lossMSE: 0.010113069787621498 lossSSIM: 0.3682336211204529
===> Epoch[2](12250/25343): Loss: 0.0996
lossMSE: 0.015339475125074387 lossSSIM: 0.403206467628479
===> Epoch[2](12300/25343): Loss: 0.1123
lossMSE: 0.008091840893030167 lossSSIM: 0.34060347080230713
===> Epoch[2](12350/25343): Loss: 0.0912
lossMSE: 0.009809421375393867 lossSSIM: 0.3875066637992859
===> Epoch[2](12400/25343): Loss: 0.1042
lossMSE: 0.009726323187351227 lossSSIM: 0.3537548780441284
===> Epoch[2](12450/25343): Loss: 0.0957
lossMSE: 0.009030739776790142 lossSSIM: 0.3885171413421631
===> Epoch[2](12500/25343): Loss: 0.1039
lossMSE: 0.007558750454336405 lossSSIM: 0.3771558403968811
===> Epoch[2](12550/25343): Loss: 0.1000
lossMSE: 0.011860024183988571 lossSSIM: 0.39343923330307007
===> Epoch[2](12600/25343): Loss: 0.1073
lossMSE: 0.011007710359990597 lossSSIM: 0.3541327714920044
===> Epoch[2](12650/25343): Loss: 0.0968

lossMSE: 0.00401780242100358 lossSSIM: 0.33447128534317017
===> Epoch[2](16300/25343): Loss: 0.0866
lossMSE: 0.008752744644880295 lossSSIM: 0.3319716453552246
===> Epoch[2](16350/25343): Loss: 0.0896
lossMSE: 0.012289045378565788 lossSSIM: 0.3367941379547119
===> Epoch[2](16400/25343): Loss: 0.0934
lossMSE: 0.0070583936758339405 lossSSIM: 0.3122689723968506
===> Epoch[2](16450/25343): Loss: 0.0834
lossMSE: 0.013435307890176773 lossSSIM: 0.33727192878723145
===> Epoch[2](16500/25343): Loss: 0.0944
lossMSE: 0.0034603651147335768 lossSSIM: 0.28243643045425415
===> Epoch[2](16550/25343): Loss: 0.0732
lossMSE: 0.014648248441517353 lossSSIM: 0.40062063932418823
===> Epoch[2](16600/25343): Loss: 0.1111
lossMSE: 0.006693067029118538 lossSSIM: 0.3493500351905823
===> Epoch[2](16650/25343): Loss: 0.0924
lossMSE: 0.007817242294549942 lossSSIM: 0.3285815119743347
===> Epoch[2](16700/25343): Loss: 0.0880


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
from torchvision import transforms, utils
def test():
    avg_psnr = 0
    with torch.no_grad():
        for iteration, batch in enumerate(dataloader):
            input, target = batch['input'].to(device), batch['output'].to(device)
                
            prediction = model(input)

            target = target.float()

            outdataloader = {'input':prediction,'output':target}
            
            print(outdataloader['input'].size(), 
                      outdataloader['output'].size())

            plt.figure()
            input_batch, label_batch = outdataloader['input'], outdataloader['output']
            batch_size = 50
            im_size = input_batch.size(2)
            label_batch=label_batch.reshape([batch_size,1,im_size,im_size])
            print(label_batch.size())
#             for img in input_batch:
#                 for t, m, s in zip(img, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]):
#                     t.mul_(s).add_(m)
                            
#             for img in label_batch:
#                 for t, m, s in zip(img, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]):
#                     t.mul_(s).add_(m)                           

            grid = utils.make_grid(input_batch).cpu()
            plt.imshow(grid.numpy().transpose((1, 2, 0)))
            plt.figure()

            grid = utils.make_grid(label_batch).cpu()
            plt.imshow(grid.numpy().transpose((1, 2, 0)))

            plt.axis('off')
            plt.ioff()
            plt.show()
            
            targetf = target[:, None]
            
            lossMSE = criterionMSE(prediction, targetf)      
            lossSSIM = -ssim(prediction, targetf)
        
            p = 0.25
            loss = p*lossMSE + (1-p)*lossSSIM
            combineLoss = p*lossMSE.item() + (1-p)*lossSSIM.item()
#             mse = criterion(prediction, target.float())

            psnr = 10 * torch.log10(1 / loss)
            avg_psnr += psnr
            if iteration == 16:
                break
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(dataloader)))

In [ ]:
test()

In [ ]:
# restart
for epoch in range(1, 5 + 1):
    train(epoch)
#     test()
#     checkpoint(epoch)

In [10]:
def train(epoch, p, windowsize):
    epoch_loss = 0
    for iteration, batch in enumerate(dataloader):
        input, target = batch['input'].to(device), batch['output'].to(device)

        optimizer.zero_grad()
        output = model(input)
        targetf = target.float()
        targetf = targetf[:, None]
        
        lossMSE = criterionMSE(output, targetf)      
        lossSSIM = 1-ssim(output, targetf, window_size=windowsize)
        
        # 0.75->0.4 after 3 epochs
        loss = p*lossMSE + (1-p)*lossSSIM
        combineLoss = p*lossMSE.item() + (1-p)*lossSSIM.item()
        
        epoch_loss = epoch_loss + combineLoss
        loss.backward()
        optimizer.step()
    
        if iteration%50 == 0:
            print("lossMSE: " + str(lossMSE.item()) +
                  " " + "lossSSIM: " + str(lossSSIM.item()))
            print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(dataloader), loss.item()))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(dataloader)))

In [ ]:
l = 0.0001
p = 0
windowsize = 4
for epoch in range(1, 20 + 1):
    if epoch%5 == 0:
        windowsize = windowsize+1
        p = p*0.8
        l = l*0.2
        if windowsize == 0:
            windowsze = 1
    optimizer = optim.Adam(model.parameters(), lr=l)
        
    train(epoch, p, windowsize)

lossMSE: 0.006996138021349907 lossSSIM: 0.34353458881378174
===> Epoch[1](0/25343): Loss: 0.3435
lossMSE: 0.0035437080077826977 lossSSIM: 0.3383822441101074
===> Epoch[1](50/25343): Loss: 0.3384
lossMSE: 0.005230362992733717 lossSSIM: 0.33881402015686035
===> Epoch[1](100/25343): Loss: 0.3388
lossMSE: 0.0065048751421272755 lossSSIM: 0.34571373462677
===> Epoch[1](150/25343): Loss: 0.3457
lossMSE: 0.005852372385561466 lossSSIM: 0.3129233121871948
===> Epoch[1](200/25343): Loss: 0.3129
lossMSE: 0.00902379211038351 lossSSIM: 0.35028302669525146
===> Epoch[1](250/25343): Loss: 0.3503
lossMSE: 0.00847490131855011 lossSSIM: 0.33797991275787354
===> Epoch[1](300/25343): Loss: 0.3380
lossMSE: 0.006889975629746914 lossSSIM: 0.3429279327392578
===> Epoch[1](350/25343): Loss: 0.3429
lossMSE: 0.016553329303860664 lossSSIM: 0.33446818590164185
===> Epoch[1](400/25343): Loss: 0.3345
lossMSE: 0.008084922097623348 lossSSIM: 0.29520106315612793
===> Epoch[1](450/25343): Loss: 0.2952
lossMSE: 0.00639685

lossMSE: 0.005720946937799454 lossSSIM: 0.31329047679901123
===> Epoch[1](4150/25343): Loss: 0.3133
lossMSE: 0.006333641707897186 lossSSIM: 0.27944743633270264
===> Epoch[1](4200/25343): Loss: 0.2794
lossMSE: 0.009462114423513412 lossSSIM: 0.31993943452835083
===> Epoch[1](4250/25343): Loss: 0.3199
lossMSE: 0.004406026564538479 lossSSIM: 0.29736000299453735
===> Epoch[1](4300/25343): Loss: 0.2974
lossMSE: 0.006743097677826881 lossSSIM: 0.29120635986328125
===> Epoch[1](4350/25343): Loss: 0.2912
lossMSE: 0.0061157396994531155 lossSSIM: 0.2786518335342407
===> Epoch[1](4400/25343): Loss: 0.2787
lossMSE: 0.005345531739294529 lossSSIM: 0.29867660999298096
===> Epoch[1](4450/25343): Loss: 0.2987
lossMSE: 0.00862218253314495 lossSSIM: 0.30137938261032104
===> Epoch[1](4500/25343): Loss: 0.3014
lossMSE: 0.006750968750566244 lossSSIM: 0.32318317890167236
===> Epoch[1](4550/25343): Loss: 0.3232
lossMSE: 0.009512131102383137 lossSSIM: 0.31996339559555054
===> Epoch[1](4600/25343): Loss: 0.3200
l

lossMSE: 0.01385992020368576 lossSSIM: 0.30841493606567383
===> Epoch[1](8300/25343): Loss: 0.3084
lossMSE: 0.008807392790913582 lossSSIM: 0.28661447763442993
===> Epoch[1](8350/25343): Loss: 0.2866
lossMSE: 0.011029642075300217 lossSSIM: 0.28498220443725586
===> Epoch[1](8400/25343): Loss: 0.2850
lossMSE: 0.01054100040346384 lossSSIM: 0.27841103076934814
===> Epoch[1](8450/25343): Loss: 0.2784
lossMSE: 0.022364668548107147 lossSSIM: 0.35585683584213257
===> Epoch[1](8500/25343): Loss: 0.3559
lossMSE: 0.016082249581813812 lossSSIM: 0.32499581575393677
===> Epoch[1](8550/25343): Loss: 0.3250
lossMSE: 0.009549826383590698 lossSSIM: 0.3067830204963684
===> Epoch[1](8600/25343): Loss: 0.3068
lossMSE: 0.013425536453723907 lossSSIM: 0.30864691734313965
===> Epoch[1](8650/25343): Loss: 0.3086
lossMSE: 0.012809328734874725 lossSSIM: 0.3050996661186218
===> Epoch[1](8700/25343): Loss: 0.3051
lossMSE: 0.016763469204306602 lossSSIM: 0.3113775849342346
===> Epoch[1](8750/25343): Loss: 0.3114
lossM

lossMSE: 0.009149076417088509 lossSSIM: 0.2832818627357483
===> Epoch[1](12450/25343): Loss: 0.2833
lossMSE: 0.013565373606979847 lossSSIM: 0.2890843152999878
===> Epoch[1](12500/25343): Loss: 0.2891
lossMSE: 0.012466657906770706 lossSSIM: 0.29401224851608276
===> Epoch[1](12550/25343): Loss: 0.2940
lossMSE: 0.012211994268000126 lossSSIM: 0.30763453245162964
===> Epoch[1](12600/25343): Loss: 0.3076
lossMSE: 0.019737839698791504 lossSSIM: 0.3094859719276428
===> Epoch[1](12650/25343): Loss: 0.3095
lossMSE: 0.015035990625619888 lossSSIM: 0.32111865282058716
===> Epoch[1](12700/25343): Loss: 0.3211
lossMSE: 0.010142306797206402 lossSSIM: 0.2709483504295349
===> Epoch[1](12750/25343): Loss: 0.2709
lossMSE: 0.011643309146165848 lossSSIM: 0.2921063303947449
===> Epoch[1](12800/25343): Loss: 0.2921
lossMSE: 0.013892359100282192 lossSSIM: 0.29252439737319946
===> Epoch[1](12850/25343): Loss: 0.2925
lossMSE: 0.010363736189901829 lossSSIM: 0.3112940192222595
===> Epoch[1](12900/25343): Loss: 0.3

lossMSE: 0.012428302317857742 lossSSIM: 0.2874920964241028
===> Epoch[1](16550/25343): Loss: 0.2875
lossMSE: 0.009322798810899258 lossSSIM: 0.2700825333595276
===> Epoch[1](16600/25343): Loss: 0.2701
lossMSE: 0.011572682298719883 lossSSIM: 0.2638641595840454
===> Epoch[1](16650/25343): Loss: 0.2639
lossMSE: 0.01928662322461605 lossSSIM: 0.26631003618240356
===> Epoch[1](16700/25343): Loss: 0.2663
lossMSE: 0.016177259385585785 lossSSIM: 0.3167310357093811
===> Epoch[1](16750/25343): Loss: 0.3167
lossMSE: 0.016652792692184448 lossSSIM: 0.28544026613235474
===> Epoch[1](16800/25343): Loss: 0.2854
lossMSE: 0.010245734825730324 lossSSIM: 0.29933297634124756
===> Epoch[1](16850/25343): Loss: 0.2993
lossMSE: 0.008426062762737274 lossSSIM: 0.3382439613342285
===> Epoch[1](16900/25343): Loss: 0.3382
lossMSE: 0.01888001710176468 lossSSIM: 0.2880871891975403
===> Epoch[1](16950/25343): Loss: 0.2881
lossMSE: 0.008551002480089664 lossSSIM: 0.26967674493789673
===> Epoch[1](17000/25343): Loss: 0.269

...

lossMSE: 0.010853693820536137 lossSSIM: 0.28387683629989624
===> Epoch[4](22900/25343): Loss: 0.2839
lossMSE: 0.014092713594436646 lossSSIM: 0.24757075309753418
===> Epoch[4](22950/25343): Loss: 0.2476
lossMSE: 0.005580798722803593 lossSSIM: 0.24459022283554077
===> Epoch[4](23000/25343): Loss: 0.2446
lossMSE: 0.0047290450893342495 lossSSIM: 0.24293804168701172
===> Epoch[4](23050/25343): Loss: 0.2429
lossMSE: 0.012091893702745438 lossSSIM: 0.29090768098831177
===> Epoch[4](23100/25343): Loss: 0.2909
lossMSE: 0.004000866785645485 lossSSIM: 0.24514412879943848
===> Epoch[4](23150/25343): Loss: 0.2451
lossMSE: 0.007059925235807896 lossSSIM: 0.29097485542297363
===> Epoch[4](23200/25343): Loss: 0.2910
lossMSE: 0.01123377401381731 lossSSIM: 0.278556764125824
===> Epoch[4](23250/25343): Loss: 0.2786
lossMSE: 0.011248597875237465 lossSSIM: 0.26646244525909424
===> Epoch[4](23300/25343): Loss: 0.2665
lossMSE: 0.004168981686234474 lossSSIM: 0.28169137239456177
===> Epoch[4](23350/25343): Loss:

lossMSE: 0.006875480525195599 lossSSIM: 0.23520386219024658
===> Epoch[5](1650/25343): Loss: 0.2352
lossMSE: 0.008952285163104534 lossSSIM: 0.29064303636550903
===> Epoch[5](1700/25343): Loss: 0.2906
lossMSE: 0.015917174518108368 lossSSIM: 0.3078082203865051
===> Epoch[5](1750/25343): Loss: 0.3078
lossMSE: 0.0036397352814674377 lossSSIM: 0.25711995363235474
===> Epoch[5](1800/25343): Loss: 0.2571
lossMSE: 0.007344708777964115 lossSSIM: 0.2274550199508667
===> Epoch[5](1850/25343): Loss: 0.2275
lossMSE: 0.008444711565971375 lossSSIM: 0.24239110946655273
===> Epoch[5](1900/25343): Loss: 0.2424
lossMSE: 0.006506847683340311 lossSSIM: 0.27891355752944946
===> Epoch[5](1950/25343): Loss: 0.2789
lossMSE: 0.009569705463945866 lossSSIM: 0.2779321074485779
===> Epoch[5](2000/25343): Loss: 0.2779
lossMSE: 0.010914876125752926 lossSSIM: 0.28530120849609375
===> Epoch[5](2050/25343): Loss: 0.2853
lossMSE: 0.005284668877720833 lossSSIM: 0.23002970218658447
===> Epoch[5](2100/25343): Loss: 0.2300
lo

lossMSE: 0.007574971299618483 lossSSIM: 0.2942546010017395
===> Epoch[5](5800/25343): Loss: 0.2943
lossMSE: 0.006769425701349974 lossSSIM: 0.2417203187942505
===> Epoch[5](5850/25343): Loss: 0.2417
lossMSE: 0.003892091102898121 lossSSIM: 0.2337561845779419
===> Epoch[5](5900/25343): Loss: 0.2338
lossMSE: 0.009684395045042038 lossSSIM: 0.3108202815055847
===> Epoch[5](5950/25343): Loss: 0.3108
lossMSE: 0.0038159803953021765 lossSSIM: 0.21976995468139648
===> Epoch[5](6000/25343): Loss: 0.2198
lossMSE: 0.00797001551836729 lossSSIM: 0.2747495174407959
===> Epoch[5](6050/25343): Loss: 0.2747
lossMSE: 0.0051012285985052586 lossSSIM: 0.29951274394989014
===> Epoch[5](6100/25343): Loss: 0.2995
lossMSE: 0.00843257550150156 lossSSIM: 0.29008060693740845
===> Epoch[5](6150/25343): Loss: 0.2901
lossMSE: 0.0044636195525527 lossSSIM: 0.24525922536849976
===> Epoch[5](6200/25343): Loss: 0.2453
lossMSE: 0.007047317922115326 lossSSIM: 0.27671921253204346
===> Epoch[5](6250/25343): Loss: 0.2767
lossMSE

lossMSE: 0.00496578449383378 lossSSIM: 0.25861507654190063
===> Epoch[5](9950/25343): Loss: 0.2586
lossMSE: 0.0073857298120856285 lossSSIM: 0.25986379384994507
===> Epoch[5](10000/25343): Loss: 0.2599
lossMSE: 0.005135920364409685 lossSSIM: 0.2464064359664917
===> Epoch[5](10050/25343): Loss: 0.2464
lossMSE: 0.00871023628860712 lossSSIM: 0.244001567363739
===> Epoch[5](10100/25343): Loss: 0.2440
lossMSE: 0.008966194465756416 lossSSIM: 0.3036285638809204
===> Epoch[5](10150/25343): Loss: 0.3036
lossMSE: 0.012774946168065071 lossSSIM: 0.31387990713119507
===> Epoch[5](10200/25343): Loss: 0.3139
lossMSE: 0.0077650099992752075 lossSSIM: 0.3212934136390686
===> Epoch[5](10250/25343): Loss: 0.3213
lossMSE: 0.00506833428516984 lossSSIM: 0.24055075645446777
===> Epoch[5](10300/25343): Loss: 0.2406
lossMSE: 0.006819423753768206 lossSSIM: 0.23857563734054565
===> Epoch[5](10350/25343): Loss: 0.2386
lossMSE: 0.007424531504511833 lossSSIM: 0.22599148750305176
===> Epoch[5](10400/25343): Loss: 0.22

lossMSE: 0.01126806903630495 lossSSIM: 0.30450183153152466
===> Epoch[5](14050/25343): Loss: 0.3045
lossMSE: 0.008418451994657516 lossSSIM: 0.31123286485671997
===> Epoch[5](14100/25343): Loss: 0.3112
lossMSE: 0.007859088480472565 lossSSIM: 0.2889865040779114
===> Epoch[5](14150/25343): Loss: 0.2890
lossMSE: 0.013717017136514187 lossSSIM: 0.27322083711624146
===> Epoch[5](14200/25343): Loss: 0.2732
lossMSE: 0.0081748366355896 lossSSIM: 0.2872501015663147
===> Epoch[5](14250/25343): Loss: 0.2873
lossMSE: 0.011711923405528069 lossSSIM: 0.33109742403030396
===> Epoch[5](14300/25343): Loss: 0.3311
lossMSE: 0.010933884419500828 lossSSIM: 0.28023189306259155
===> Epoch[5](14350/25343): Loss: 0.2802
lossMSE: 0.005528024863451719 lossSSIM: 0.22963815927505493
===> Epoch[5](14400/25343): Loss: 0.2296
lossMSE: 0.004766278900206089 lossSSIM: 0.21990883350372314
===> Epoch[5](14450/25343): Loss: 0.2199
lossMSE: 0.005511588416993618 lossSSIM: 0.288055419921875
===> Epoch[5](14500/25343): Loss: 0.28